**CC7221:** Reconocimiento Visual con Deep Learning

**Prof:** José M. Saavedra Rondo 

**Teacher Assistants:** Pablo Torres, Cristóbal Loyola 

**Members:** Humberto Rodrigues, David Rojas

# Abstract

La segmentación de imágenes es uno de los campos de la visión por computadora que ha visto grandes avances en los últimos tiempos, impulsado por sus diversos usos, desde carros con sistemas de conducción automatizado, hasta software de monitoreo de multitudes. En este trabajo se abordará el modelo `DeepLab` con la finalidad de detectar espermatozoides humanos en una imagen y las distintas partes que lo componen.

# Introducción

El dataset original utilizado para este problema constaba de 14 imágenes para train y 6 para test, por cada imagen se tenía una máscara correspondiente a cada parte del espermatozoide (cabeza, cuerpo y cola).

Al tener tan pocos elementos este dataset, fue necesario usar técnicas de `data augmentation`, tras las cuales se obtuvo un dataset total de train de 1665 imgenes y 444 imágenes para test, ambas con sus respectivas máscaras.

Una vez completada la generación del dataset a utilizar se procedió a la construcción de los modelos utilizando `pytorch`.

El problema a resolver es un problema de Clasificación con 4 clases `['__background__', 'head', 'body', 'tail'].

# Desarrollo

## Manejo de Data

Como se mencionó anteriormente se utilizaron técnicas de `data augmentation` para incrementar el volumen de datos para ambos datasets a utilizar. Particularmente se aplicaron las siguientes transformaciones sobre las imágenes y máscaras originales para así obtener nuevas instancias de representación.

1. Las imágenes originales fueron rotadas hasta los 180° en sentido antihorario, usando pasos de 5°
2. Se aplicó un flip horizontal a las imágenes originales y luego todo el paso `1`
3. Se aplicó un flip vertical a las imágenes originales y luego todo el paso `1`

En cuanto al tamaño de las imágenes a ambas se le aplicó un resize de `(580x780)`. Durante los experimentos que utilizaron un `backbone` preentrenado, las imágenes a color fueron normalizadas utilizando `mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]` debido a que `pytorch` así lo amerita a la hora de utilizar sus modelos.

## Modelos

El modelo utilizado fue [DeeplabV3](https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/), este modelo tiene principalmente dos componentes:

- Un backbone ResNet (en este caso ResNet-50)

- Un clasificador (Head) con el número de clases que se esperan predecir

Nuestros experimentos se basaron en utilizar versiones preentrenadas o no de dicho backbone, mientras que los pesos del clasificador siempre eran aprendidos desde cero, de acuerdo al número de clases que amerita este problema.

Cabe destacar que los modelos preentrenados utilizaron el dataset `COCO train2017` con 20 clases presentes en el `PASCAL VOC dataset`. El objetivo de esto es comprobar si existe conocimiento que pueda ayudar a solucionar el problema de segmentación de imágenes a pesar de que estos fueron entrenados en un contexto distinto y compararlos con modelos con pesos limpios que aprendan directamente del contexto de nuestro problema actual.

En esta sección vamos a mostrar los cuatro experimentos principales que se realizaron. Ciertos parámetros en común para los experimentos fueron:

- optimizador: `AdamW` con un `learn_rate=1e-3`

- loss: `CrossEntropyLoss`

# Resultados

## Experimento 1

En este exprimento se utilizó el backbone preentrenado, las imágenes se normalizaron, se utilizó un `batch_size=10` y `epochs=25`. Debido al tamaño de las imágenes nos fue imposible contemplar `batch_size` de mayor tamaño.

### Gráfico de la Loss

![report_imgs/ex1_loss.png](report_imgs/ex1_loss.png)

### Valores promedio obtenidos

loss: 0.07586 | IoU: 0.31242 | Dice: 0.02615

### Algunos ejemplos de lo obtenido
Máscaras de background, head, body y tail en ese orden

![report_imgs/ex1.png](report_imgs/ex1.png)

## Experimento 2

En este exprimento se utilizó el backbone sin preentrenar, las imágenes no se normalizaron, se utilizó un `batch_size=10` y `epochs=25`.

### Gráfico de la Loss

![report_imgs/ex2_loss.png](report_imgs/ex2_loss.png)

### Valores promedio obtenidos

loss: 0.35263 | IoU: 0.00000 | Dice: 0.02491

### Algunos ejemplos de lo obtenido
Máscaras de background, head, body y tail en ese orden

![report_imgs/ex2.png](report_imgs/ex2.png)

## Experimento 3

En este exprimento se utilizó el backbone preentrenado, las imágenes se normalizaron, se utilizó un `batch_size=5` y `epochs=10`.

### Gráfico de la Loss

### Valores promedio obtenidos

loss: 0.07198 | IoU: 0.32160 | Dice: 0.02613

### Algunos ejemplos de lo obtenido
Máscaras de background, head, body y tail en ese orden

![report_imgs/ex3.png](report_imgs/ex3.png)

## Experimento 4

En este exprimento se utilizó el backbone sin preentrenar, las imágenes no se normalizaron, se utilizó un `batch_size=5` y `epochs=10`.

### Gráfico de la Loss

### Valores promedio obtenidos

loss: 0.81401 | IoU: 0.00000 | Dice: 0.02474

### Algunos ejemplos de lo obtenido
Máscaras de background, head, body y tail en ese orden

![report_imgs/ex4.png](report_imgs/ex4.png)

## Otros Experimentos

Adicional a los elementos mostrados en los experimentos anteriores, se realizó otro conjunto de experimentos variando los siguientes aspectos:

- Modelo preentrenado **sin** utilizar imágenes normalizadas

- Resize de imágenes a (600x600), seguido de un centerCrop(400x400)

- Usar un compound Loss definido como `(0.8 * CrossEntropyLoss) + (0.2 * DiceLoss)`

- Usar optimizador `Adadelta` con `learn_rate=1e-4`

Estos resultados no fueron mostrados en detalle, debido a que no ofrecian una mejora significativa, adicionalmente tenían problemas para representar las clases con poca densidad de pixeles (`tails`).

# Conclusiones y Análisis de Resultados

Basándonos en los resultados obtenidos de nuestros exprimentos, podemos concluir lo siguiente:

- Existe información que resuelve el problema de segmentación de imágenes en los backbone de los modelos preentrenados (al menos en `pytorch`) incluso si estos fueron entrenados usando un contexto distinto a aquiel con el que se esta probando

- Si bien en los modelos "limpios" no se obtienen buenos resultados, se puede apreciar en el modelo de 25 épocas que empiezan a aparecer detecciones de segmentos dentro de las imágenes, por lo que se puede concluir que con un volumen de datos superior y un mayor tiempo de entrenamiento se podría llegar a lograr un conocimiento similar al que se encuentra en los modelos preentrenados.

- CrossEntropyLoss, por si sola no parece ser la función de loss más adecuada para este problema, si bien es una de las más comunmente utilizadas. Esto debido a los bajos valores que se obtienen para la función de loss en base a otras métricas (iou o diceLoss).

- Finalmente, el enfoque de utilizar `batch_size=5` y `epochs=10` dan resultados bastante similares a su homólogo `batch_size=10` y `epochs=25`, en considerablemente menos tiempo de entrenamiento (~50%).

Como trabajo futuro se plantea una comparativa entre los otros dos modelos ofrecidos por `pytorch` para la segmentación de imágenes que son FCN-50, LR-ASPP.


Todos los experimentos, código fuente y manipulación de la data utilizado para esta tarea se encuentra en este [repositorio](https://github.com/davidrojas0791/VR_DL_4).